In [11]:
import multiprocessing
import numpy as np
import os

from dlp.dataset import Dataset
from dlp.visualizer import MAP_SIZE, Visualizer
from math import cos, sin
from parksim.trajectory_predict.data_processing.utils import TransformerDataProcessor
from pathlib import Path
from tqdm import tqdm
from typing import Tuple
import pickle as pkl
from vehicle import Vehicle, Location, Size3d, get_box_pts_from_center_heading
from math import pi

- Vehicle对象存储跟输入输出相关的所有的东西

In [12]:
stride = 10
history = 5
future = 5
cpu_count = os.cpu_count()
ds = Dataset()
ds.load(f"{Path.home()}/dlp-dataset/data/"+ "DJI_0012")
vis = Visualizer(ds)
scene_token = ds.list_scenes()[0]
scene = ds.get('scene', scene_token)
agents_token = scene['agents']
agent_token2idx = {agent_token: str(idx) for idx, agent_token in enumerate(agents_token)}

- 获取局部地图四个角点在全局坐标系的位置
- 筛选出在局部地图内的agent
- 筛选障碍物
- 筛选空和占用车库
- 目标点 车库或道路

In [13]:
def is_overlap(rect1, rect2):
    # 计算每个矩形的最小和最大x和y坐标
    min_x1, min_y1 = np.min(rect1, axis=0)
    max_x1, max_y1 = np.max(rect1, axis=0)
    min_x2, min_y2 = np.min(rect2, axis=0)
    max_x2, max_y2 = np.max(rect2, axis=0)

    # 检查两个矩形是否重叠
    if min_x1 > max_x2 or min_x2 > max_x1 or min_y1 > max_y2 or min_y2 > max_y1:
        return False
    else:
        return True

In [14]:
def get_data_for_instance(inst_token: str, inst_idx: int, frame_token: str, extractor: TransformerDataProcessor, ds: Dataset, agent_token2idx: dict, vis: Visualizer):
    veh = Vehicle()
    curr_instance = ds.get('instance', inst_token)
    agent_token = curr_instance['agent_token']
    veh.id = agent_token2idx[agent_token]
    veh.location = Location(curr_instance["coords"][0], curr_instance["coords"][1], None)
    veh.speed_heading = curr_instance["heading"] / pi * 180
    veh.size = Size3d(1.8, 3.6, 1.5)
    veh.safe_size = Size3d(2.0, 3.8, 1.5)
    veh.update_poly_box_and_realworld_4_vertices()
    veh.update_safe_poly_box()

    curr_pose = np.array([curr_instance['coords'][0],
                         curr_instance['coords'][1], curr_instance['heading']])
    rot = np.array([[cos(-curr_pose[2]), -sin(-curr_pose[2])], [sin(-curr_pose[2]), cos(-curr_pose[2])]])
    global_map_size = MAP_SIZE
    # 以当前车的位置为坐标系，计算局部地图窗口4个顶点的坐标，窗口大小为inst_ctr_size
    inst_ctr_size = 40
    map_global_4_vertices = get_box_pts_from_center_heading(length=inst_ctr_size, width=inst_ctr_size, xc=veh.location.x, yc=veh.location.y, heading=veh.speed_heading)
    map_global_4_vertices = np.array(map_global_4_vertices)
    # 将局部地图超出全局地图的部分裁剪掉
    map_global_4_vertices[:, 0] = np.clip(map_global_4_vertices[:, 0], 0, global_map_size['x'])
    map_global_4_vertices[:, 1] = np.clip(map_global_4_vertices[:, 1], 0, global_map_size['y'])
    # 将map_global_4_vertices转为以当前车为坐标系的坐标
    map_local_4_vertices = map_global_4_vertices.copy()
    map_local_4_vertices = np.dot(rot, (map_local_4_vertices - curr_pose[:2]).T).T
    veh.map_local_4_vertices = map_local_4_vertices

    curr_frame = ds.get('frame', frame_token)
    all_curr_instance_tokens = curr_frame['instances']
    selected_instance_tokens=[]
    for temp_instance_token in all_curr_instance_tokens:
        temp_instance = ds.get('instance', temp_instance_token)
        # 判断当前instance是否在局部地图窗口内
        # 转化为以当前车为坐标系的坐标
        temp_instance_local_coord = np.dot(rot, (np.array(temp_instance['coords']) - curr_pose[:2]).T).T
        if np.all(np.fabs(temp_instance_local_coord) < inst_ctr_size/2):
            selected_instance_tokens.append(temp_instance_token)
    for temp_instance_token in selected_instance_tokens:
        # 获取车辆的历史轨迹
        temp_bv_history = []
        temp_instance = ds.get('instance', temp_instance_token)
        for timestep in range(0, stride*history, stride):
            temp_history_instance = ds.get_agent_past(temp_instance_token, timestep)[0]
            temp_history_instance_local_coord = np.dot(rot, (np.array(temp_history_instance['coords']) - curr_pose[:2]).T).T

            temp_history_instance_local_heading = temp_history_instance['heading'] - curr_pose[2]
            temp_history_instance_local_pose = np.concatenate([temp_history_instance_local_coord, [temp_history_instance_local_heading]])
            temp_bv_history.append(temp_history_instance_local_pose)
        temp_bv_history.reverse()
        temp_bv_history = np.array(temp_bv_history)
        if agent_token == temp_instance['agent_token']:
            veh.av_history = temp_bv_history
        else:
            veh.bvs_history_list.append(temp_bv_history)
        # 获取车辆的未来轨迹
        temp_bv_future = []
        for timestep in range(stride, stride*future+1, stride):
            temp_future_instance = ds.get_agent_future(temp_instance_token, timestep)[-1]
            temp_future_instance_local_coord = np.dot(rot, (np.array(temp_future_instance['coords']) - curr_pose[:2]).T).T
            temp_future_instance_local_heading = temp_future_instance['heading'] - curr_pose[2]
            temp_future_instance_local_pose = np.concatenate([temp_future_instance_local_coord, [temp_future_instance_local_heading]])
            temp_bv_future.append(temp_future_instance_local_pose)
        temp_bv_future = np.array(temp_bv_future)
        if agent_token == temp_instance['agent_token']:
            veh.av_future = temp_bv_future
        else:
            veh.bvs_future_list.append(temp_bv_future)

    # 获取局部地图窗口内的障碍物
    scene_token = ds.list_scenes()[0]
    scene = ds.get('scene', scene_token)
    all_obstacle_tokens = scene['obstacles']
    selected_obstacle_tokens=[]
    for temp_obstacle_token in all_obstacle_tokens:
        temp_obstacle = ds.get('obstacle', temp_obstacle_token)
        # temp_obstacle_local_coord = np.dot(np.array(temp_obstacle['coords']) - curr_pose[:2], rot)
        temp_obstacle_local_coord = np.dot(rot, (np.array(temp_obstacle['coords']) - curr_pose[:2]).T).T
        if np.all(np.fabs(temp_obstacle_local_coord) < inst_ctr_size/2):
            selected_obstacle_tokens.append(temp_obstacle_token)
    for temp_obstacle_token in selected_obstacle_tokens:
        temp_obstacle = ds.get('obstacle', temp_obstacle_token)
        # temp_obstacle_local_coord = np.dot(np.array(temp_obstacle['coords']) - curr_pose[:2], rot)
        temp_obstacle_local_coord = np.dot(rot, (np.array(temp_obstacle['coords']) - curr_pose[:2]).T).T
        temp_obstacle_local_heading = temp_obstacle['heading'] - curr_pose[2]
        temp_obstacle_local_pose = np.concatenate([temp_obstacle_local_coord, [temp_obstacle_local_heading]])
        temp_obstacle_4_vertices = get_box_pts_from_center_heading(length=temp_obstacle['size'][0], width=temp_obstacle['size'][1], xc=temp_obstacle_local_pose[0], yc=temp_obstacle_local_pose[1], heading=temp_obstacle_local_pose[2] / pi * 180)
        temp_obstacle_4_vertices = np.array(temp_obstacle_4_vertices)
        veh.obstacle_4_vertices_list.append(temp_obstacle_4_vertices)
    
    # 获取局部地图窗口内的路点
    all_waypoints = np.empty((0, 2))
    for _, points in vis.waypoints.items():
        all_waypoints = np.append(all_waypoints, points, axis=0)
    # all_waypoints_local_coord = np.dot(all_waypoints - curr_pose[:2], rot)
    all_waypoints_local_coord = np.dot(rot, (all_waypoints - curr_pose[:2]).T).T
    selected_waypoints_idx = np.where(np.all(np.fabs(all_waypoints_local_coord) < inst_ctr_size/2, axis=1))[0]
    veh.waypoints_local_coord = all_waypoints_local_coord[selected_waypoints_idx]

    # 获取局部地图窗口内的停车位信息
    all_parking_4_vertices = vis.parking_spaces.iloc[:, -8:].values.reshape(-1, 2)
    all_parking_4_vertices_local_coord = np.dot(rot, (all_parking_4_vertices - curr_pose[:2]).T).T.reshape(-1, 4, 2)
    selected_parking_idx = np.where( np.all(np.all(np.fabs(all_parking_4_vertices_local_coord) < inst_ctr_size/2, axis=2), axis=1) )[0]
    veh.parkings_4_vertices_local_coord = all_parking_4_vertices_local_coord[selected_parking_idx]

    # AV目标点
    img_frame = extractor.vis.plot_frame(frame_token)
    image_feature = extractor.vis.inst_centric(img_frame, inst_token)
    global_intent_pose = extractor.get_intent_pose(inst_token=inst_token, inst_centric_view=image_feature)
    local_intent_coords = np.dot(rot, global_intent_pose[:2]-curr_pose[:2])
    local_intent_pose = np.array(
        [local_intent_coords[0], local_intent_coords[1], global_intent_pose[2]-curr_pose[2]])
    veh.av_intent_pose = local_intent_pose    
    # 保存veh为pickle文件

In [15]:
extractor = TransformerDataProcessor(ds=ds, tail_size=10)
all_frames = []
frame_token = scene['first_frame']
while frame_token:
    all_frames.append(frame_token)
    frame = ds.get('frame', frame_token)
    frame_token = frame['next']
for frame_idx in tqdm(range(stride*history, len(all_frames) - stride*future, stride)):
    frame_token = all_frames[frame_idx]
    all_instance_tokens, all_instance_indices = extractor.filter_instances(frame_token, stride, history, future)
    num_insts = len(all_instance_tokens)
    # with multiprocessing.Pool(processes=cpu_count) as pool:
    #     inputs = zip(all_instance_tokens, all_instance_indices, [frame_token]*num_insts, [extractor]*num_insts, [ds]*num_insts, [agent_token2idx]*num_insts, [vis]*num_insts)
    #     results = pool.starmap(get_data_for_instance, inputs)
    for inst_idx in range(num_insts):
        get_data_for_instance(all_instance_tokens[inst_idx], all_instance_indices[inst_idx], frame_token, extractor, ds, agent_token2idx, vis)
    # get_data_for_instance(all_instance_tokens[0], all_instance_indices[0], frame_token, extractor, ds, agent_token2idx, vis)

  1%|          | 11/1121 [00:31<53:23,  2.89s/it]


KeyboardInterrupt: 

In [ ]:
for timestep in range(stride, stride*history+1, stride):
    print(timestep)

In [ ]:
np.concatenate([np.array([1,2,3]), ([4])])